<header style=" text-align: center; color:blue; font-size:45px">Order Imbalance Based Strategy</header>

<h1 style="color:purple">Author</h1>

  
- Name:                      
- Student ID: 


## Introduction

Many studies have been conducted to describe the relationship between the trade activity(volume) and the price change. Ask (Sell) and Bid (Buy) orders on the border book might signal the direction of market movement. To  model the possible pattern between the order imbalance (between ask and bid sides) and price change, we will build new predictors and check the performance of multiple linear regression in paper trading (backtesting with historical data). 

In this project, you will finish 4 tasks independently. It is not a copy-and-paste project. But if you can complete this project by yourself, it shows that you have passed the entry-level as a data analyst. 

## Data

Totally we have 10 days' data

In [1]:
import pandas as pd
import numpy as np
import os, math, pathlib
from os import listdir
from multiprocessing import Process
import warnings
warnings.filterwarnings('ignore')

In [2]:
stockpath = 'data/'
inputfilename = listdir(stockpath) # get all file names in this folder
inputfilename = sorted(inputfilename)[:] # sort file names 
number = len(inputfilename)
print(number)
print(inputfilename)

10
['20201221_rb2105.csv', '20201222_rb2105.csv', '20201223_rb2105.csv', '20201224_rb2105.csv', '20201225_rb2105.csv', '20210104_rb2105.csv', '20210105_rb2105.csv', '20210106_rb2105.csv', '20210107_rb2105.csv', '20210108_rb2105.csv']


We list file names for these 10 days.  Notice that they are in time order. 

In [18]:
inputfilename[0],inputfilename[-1]

('20201221_rb2105.csv', '20210108_rb2105.csv')

For example, we can read the oldest data as follows.

In [3]:
n=10
data=pd.read_csv(stockpath+inputfilename[-n],index_col=0)
data.head()

,AskPrice,BidPrice,AskVolume,BidVolume
2020-12-21 09:00:00.500,4445.0,4419.0,467.0,87.0
2020-12-21 09:00:01.000,4445.0,4444.0,88.0,67.0
2020-12-21 09:00:01.500,4433.0,4420.0,6.0,462.0
2020-12-21 09:00:02.000,4430.0,4428.0,40.0,9.0
2020-12-21 09:00:02.500,4427.0,4426.0,1.0,81.0


We  read data on Dec 21 , 2020, which has 4 columns. 

## Task 1 - Prepare the Train and Test (10%)

Please use the first 7 days' data to prepare a training data and the remaining (more recent) 3 days' data as the test data. 

## Task 2: Build Features (50%)
For both training and test data, build predictors and response

First build mid price
$$Mid_t=\frac{AskPrice_t+BidPrice_t}{2}$$

The frequency of the data is 0.5 second. We want to predict the price change in 10 seconds. Build Y variable 
$$
Y_t=\frac{Mid_{t+1}+Mid_{t+2}+\ldots+Mid_{t+20}}{20}-Mid_t
$$
That is, we calculate the average changes of prices for the next 10 seconds.  

Next we build predictors 

#### Volume Order ImBalance

First Build 
$$
OI_t=\delta V_t^B-\delta V_t^A
$$

where

$$
\delta V_t^B=\left\{\begin{array}{cl}0, & \mbox { if } P_t^B<P_{t-1}^B\\V_t^B-V_{t-1}^B, & \mbox { if } P_t^B=P_{t-1}^B\\V_{t}^B, & \mbox { if } P_t^B>P_{t-1}^B\end{array}\right.
$$
and 
$$
\delta V_t^A=\left\{\begin{array}{cl}V_t^A, & \mbox { if } P_t^A<P_{t-1}^A\\V_t^A-V_{t-1}^A, & \mbox { if } P_t^A=P_{t-1}^A\\0, & \mbox { if } P_t^A>P_{t-1}^A\end{array}\right.
$$

where $V_t^A,V_t^B$ are ask and bid volumes, $P_t^A,P_t^B$ are ask and bid prices

Use shift(2),shift(4) ... operators, we  build $OI_{t-2},OI_{t-4},OI_{t-6},OI_{t-8}, OI_{t-10}$, totally 6 predictors including $OI_t$

## Task 3: Make Multiple Linear Regression (20%)

Build multiple linear regression to predict $Y_t$ with $[OI_t, OI_{t-2},OI_{t-4},OI_{t-6},OI_{t-8}, OI_{t-10}]$ in training data. 

Check if there exists multicollinearity and check the residuals to see if the assumptions are satisfied. Discuss your results. 

Using the model you build with training data, get predictions in both training and test data. Comparing Adjusted $R^2$ of the model in both train and test datasets, how to evaluate your model? Overfitting? Good Signal?

## Task 4: Nonlinear regression  and Subset Slection (20%) 

Build interaction terms and square terms  out of $[OI_t, OI_{t-2},OI_{t-4},OI_{t-6},OI_{t-8}, OI_{t-10}]$.   Apply best subset selection with indirected criterion (Adjusted Rsquare).  Evaluate your model in Test dataset. 